In [15]:
import os
import pandas as pd
import numpy as np
from glob import glob
import csv

In [16]:
raw_path='./data/'
output_dir='./output/'
template_dir='./templates/'
cleaned_dir='./clean_output/'

In [17]:
template1=os.path.join(template_dir,'ant_block_1_template.csv')
template2=os.path.join(template_dir,'ant_block_2_template.csv')
temp_1_df=pd.read_csv(template1,dtype=str)
temp_2_df=pd.read_csv(template2,dtype=str)

r1=map(str,[0,0,800,400,400,400,400,0,400,800])
r2=map(str,[400,800,0,800,800,400,400,800,0,0])

for idx,file in enumerate([f for f in glob(os.path.join(raw_path,'*'))]):
    try:
        raw=pd.read_csv(file,dtype=str)
        ant_r=raw.loc[raw['blockcode'] == 'ANT_R']
        for subj,subj_df in ant_r.groupby(['subject']):
            subj_df_compiled=pd.DataFrame()
            for block, block_df in subj_df.groupby(['values.blockcount']):
                if (block_df['values.CT_ISI'].values[0:10] == r1).all():
                    block_df_compiled=pd.merge(block_df.reset_index(),temp_1_df,left_index=True,right_index=True,how="outer")
                else:
                    block_df_compiled=pd.merge(block_df.reset_index(),temp_2_df,left_index=True,right_index=True,how="outer")
                subj_df_compiled=subj_df_compiled.append(block_df_compiled)
            try:
                subj_df_compiled.to_csv(os.path.join(output_dir, "batchno_%s_subj_%03d.csv"%(idx+1,int(subj))))
            except:
                subj_df_compiled.to_csv(os.path.join(output_dir, "batchno_%s_subj_%s.csv"%(idx+1,subj)))
    except:
        continue

/Users/cdla/Documents/scripts/miniconda2/lib/python2.7/site-packages/ipykernel_launcher.py:16: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  app.launch_new_instance()


In [19]:
all_data=[[]]
all_data_corr=[[]]
all_data_acc=[[]]
for batch_csv in sorted(glob(os.path.join(output_dir,'batch*.csv'))):
     try: 
#        print subj
        subj=os.path.basename(batch_csv[:-4])
        subj_df=pd.read_csv(batch_csv)
        #Data clean up
        date=subj_df['date'].unique()[0]
        subjid=subj_df['subject'].unique()[0]
        subj_df.loc[subj_df['cue_condition'].str.contains('.invalidspatial'),'cue_condition']='trial.invalid'
        subj_df.loc[subj_df['cue_condition'].str.contains('.validspatial'),'cue_condition']='trial.validspatial'
        subj_df.loc[subj_df['cue_condition'] == 'trial.nocue','cue_target_interval']='NaN'
        results=pd.DataFrame()
        
        data=subj_df[['subject','date','location_congruence_val','flanker_congruence_val','cue_target_interval','cue_condition','latency','values.correct']]
        data.to_csv(os.path.join(cleaned_dir,subj + '_cleaned.csv'),index=False)
        
        
        cols=['subjid']+['date']
        subj_vals=[subjid]+[date]
        subj_vals_corr=[subjid]+[date]
        subj_acc=[subjid]+[date]
        
        for bygroup,bygroup_df in subj_df.groupby(['location_congruence_val','flanker_congruence_val','cue_target_interval','cue_condition']):
            cols.append('loc-con-%s_flank-con-%s_cue-int-%s_cue-cond-%s'%(bygroup[0],bygroup[1],bygroup[2],bygroup[3]))
            subj_vals.append(bygroup_df['latency'].mean())
            subj_vals_corr.append(bygroup_df.loc[bygroup_df['values.correct'] == 1,'latency'].mean())
            subj_acc.append(bygroup_df['values.correct'].mean())
     except:
         print subj," did not work"
         continue        
        
        
     all_data[0]=cols
     all_data.append(subj_vals)
 
        
     all_data_corr[0]=cols
     all_data_corr.append(subj_vals_corr)
        
     all_data_acc[0]=cols
     all_data_acc.append(subj_acc)


with open('ant-r_summary_latency.csv','wb') as f:
    writer = csv.writer(f)
    writer.writerows(all_data)
    
with open('ant-r_summary_latency_corr.csv','wb') as f:
    writer = csv.writer(f)
    writer.writerows(all_data_corr)
    
with open('ant-r_summary_acc.csv','wb') as f:
    writer = csv.writer(f)
    writer.writerows(all_data_acc)

    

batchno_2_subj_663  did not work
